In [1]:
!pip install -U transformers huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 99.6 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 81.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [2]:
from IPython.display import HTML, Javascript, display

def restart_kernel_and_run_all_cells():
    display(HTML('''
        <script>
            // Restart the kernel
            IPython.notebook.kernel.restart();
            // Wait for a short period for the kernel to restart, then execute all cells
            setTimeout(function(){
                IPython.notebook.execute_all_cells();
            }, 10000); // 10000 milliseconds = 10 seconds. Adjust as needed.
        </script>
    '''))

# Call the function to restart the kernel and run all cells
restart_kernel_and_run_all_cells()

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ancient/sample_submission.csv
/kaggle/input/ancient/train.csv
/kaggle/input/ancient/test.csv


In [4]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [5]:
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F

2025-10-19 09:18:06.852027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760865486.874473     123 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760865486.881213     123 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
def load_data(csv_path, text_col='text', label_col='label'):
    """Load data from CSV"""
    df = pd.read_csv(csv_path)
    return df

def create_dataset(df, text_col='text', label_col='label', id_col='id'):
    """Convert pandas dataframe to HuggingFace dataset"""
    data_dict = {
        'text': df[text_col].tolist(),
        'label': df[label_col].tolist()
    }
    if id_col and id_col in df.columns:
        data_dict['id'] = df[id_col].tolist()
    
    dataset = Dataset.from_dict(data_dict)
    return dataset

def tokenize_function(examples, tokenizer, max_length=512):
    """Tokenize texts"""
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

def compute_metrics(eval_pred):
    """Compute metrics for evaluation"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)
    precision = precision_score(labels, predictions, average='weighted', zero_division=0)
    recall = recall_score(labels, predictions, average='weighted', zero_division=0)
    
    return {
        'accuracy': accuracy,
        'f1_macro': f1_macro,
        'precision': precision,
        'recall': recall
    }

In [10]:
class FocalLoss(torch.nn.Module):
    """Focal Loss for handling class imbalance"""
    
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        """
        Args:
            alpha: Weighting factor in range (0,1) to balance classes
            gamma: Exponent of the modulating factor (1 - p_t) to focus on hard examples
            reduction: 'mean', 'sum', or 'none'
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        p_t = torch.exp(-ce_loss)
        focal_loss = (1 - p_t) ** self.gamma * ce_loss
        
        if self.alpha is not None:
            if isinstance(self.alpha, (float, int)):
                alpha_t = self.alpha
            else:
                alpha_t = self.alpha.gather(0, targets)
            focal_loss = alpha_t * focal_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# class FocalLossTrainer(Trainer):
#     """Custom Trainer with Focal Loss"""
    
#     def __init__(self, alpha=None, gamma=2.0, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.focal_loss = FocalLoss(alpha=alpha, gamma=gamma)
    
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.logits
        
#         loss = self.focal_loss(logits, labels)
        
#         return (loss, outputs) if return_outputs else loss
class FocalLossTrainer(Trainer):
    """Custom Trainer with Focal Loss"""
    
    def __init__(self, alpha=None, gamma=2.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal_loss = FocalLoss(alpha=alpha, gamma=gamma)
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        loss = self.focal_loss(logits, labels)
        
        return (loss, outputs) if return_outputs else loss


In [11]:
def fine_tune_model_with_focal_loss(
    model_name,
    train_dataset,
    eval_dataset,
    tokenizer,
    num_labels,
    output_dir,
    num_epochs=3,
    batch_size=16,
    learning_rate=2e-5,
    gamma=2.0,
    class_weights=None
):
    """Fine-tune model with Focal Loss (handles imbalanced data better)"""
    
    print(f"\n{'='*70}")
    print(f"Fine-tuning {model_name} WITH FOCAL LOSS")
    print(f"{'='*70}\n")
    
    # Tokenize datasets
    train_dataset = train_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=['text', 'id'] if 'id' in train_dataset.column_names else ['text']
    )
    eval_dataset = eval_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=['text', 'id'] if 'id' in eval_dataset.column_names else ['text']
    )
    
    print("✓ Tokenization complete\n")
    torch.cuda.empty_cache()
    
    # Check GPU
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"✓ GPU Available: {torch.cuda.get_device_name(0)}")
        print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB\n")
    else:
        device = torch.device('cpu')
        print(f"✗ GPU NOT available, using CPU\n")
    
    # Load model
    print("Loading model...")
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels
    )
    
    model.to(device)
    model.config.pad_token_id = tokenizer.pad_token_id
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}\n")
    
    # Prepare alpha weights for Focal Loss
    alpha = None
    if class_weights is not None:
        alpha = torch.tensor(class_weights, device=device)
        print(f"Using class weights for Focal Loss\n")
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"{output_dir}/{model_name.split('/')[-1]}_focal_loss",
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,
        eval_strategy='epoch',
        save_strategy='epoch',
        learning_rate=learning_rate,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        fp16=torch.cuda.is_available(),
        gradient_accumulation_steps=2,
        optim='adamw_torch',
        report_to='none'
    )
    
    # Trainer with Focal Loss
    trainer = FocalLossTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        processing_class=tokenizer,
        alpha=alpha,
        gamma=gamma
    )
    
    # Train
    print(f"Starting training with Focal Loss (gamma={gamma})...\n")
    trainer.train()
    
    # Save model
    model.save_pretrained(f"{output_dir}/{model_name.split('/')[-1]}_focal_loss_final")
    
    print(f"\n✓ Model saved to {output_dir}/{model_name.split('/')[-1]}_focal_loss_final")
    
    return model, trainer

In [12]:
if __name__ == "__main__":
    # Configuration
    CSV_PATH = '/kaggle/input/ancient/train.csv'
    TEXT_COL = 'text'
    LABEL_COL = 'label'
    ID_COL = 'id'
    NUM_LABELS = 15
    OUTPUT_DIR = '/kaggle/working/models'
    MODEL_NAME = 'distilbert/distilbert-base-multilingual-cased'
    
    # Load data
    print("Loading data...")
    df = load_data(CSV_PATH, TEXT_COL, LABEL_COL)
    print(f"Dataset shape: {df.shape}")
    print(f"Label distribution:\n{df[LABEL_COL].value_counts()}\n")
    
    # Load tokenizer
    base_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    # Convert to dataset
    full_dataset = create_dataset(df, TEXT_COL, LABEL_COL, ID_COL)
    
    # Split into train and evaluation
    split_dataset = full_dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset = split_dataset['train']
    eval_dataset = split_dataset['test']
    
    print(f"Train samples: {len(train_dataset)}")
    print(f"Eval samples: {len(eval_dataset)}\n")
    
    # Calculate class weights for Focal Loss
    label_counts = df[LABEL_COL].value_counts().sort_index()
    total_samples = len(df)
    class_weights = [total_samples / (NUM_LABELS * count) for count in label_counts]
    print(f"Class weights for Focal Loss: {class_weights}\n")
    
    try:
        model_v2, trainer_v2 = fine_tune_model_with_focal_loss(
            model_name=MODEL_NAME,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=base_tokenizer,
            num_labels=NUM_LABELS,
            output_dir=OUTPUT_DIR,
            num_epochs=6,
            batch_size=16,
            learning_rate=2e-5,
            gamma=2.0,
            class_weights=class_weights
        )
        print("\n" + "="*70)
        print("✓ Version 2 (Focal Loss) Training Completed Successfully!")
        print("="*70)
    except Exception as e:
        print(f"\n✗ Error during Version 2 training: {str(e)}")
        import traceback
        traceback.print_exc()

Loading data...
Dataset shape: (119656, 3)
Label distribution:
label
7     35283
8     18901
5     16871
0      9104
6      7297
3      5942
1      5839
10     5643
4      5184
2      4315
11     2538
9      2426
12      169
13      142
14        2
Name: count, dtype: int64

Train samples: 95724
Eval samples: 23932

Class weights for Focal Loss: [0.8762155828939661, 1.366170006279614, 1.8486828891463887, 1.3424884999439022, 1.5387860082304528, 0.4728271392725189, 1.0931981179480152, 0.22608810664248127, 0.4220446889935277, 3.288156086837043, 1.4136215960777365, 3.1430522721302863, 47.20157790927022, 56.17652582159624, 3988.5333333333333]


Fine-tuning distilbert/distilbert-base-multilingual-cased WITH FOCAL LOSS



Map:   0%|          | 0/95724 [00:00<?, ? examples/s]

Map:   0%|          | 0/23932 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Tokenization complete

✓ GPU Available: Tesla T4
✓ GPU Memory: 15.83 GB

Loading model...
Total parameters: 135,336,207
Trainable parameters: 135,336,207

Using class weights for Focal Loss

Starting training with Focal Loss (gamma=2.0)...



Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision,Recall
1,5.249400,1.435677,0.440498,0.337746,0.515393,0.440498
2,1.421800,1.297715,0.503259,0.407854,0.589233,0.503259
3,1.188300,1.210757,0.531548,0.429966,0.606989,0.531548
4,0.902500,1.252524,0.543958,0.441927,0.612338,0.543958
5,0.786700,1.267177,0.559920,0.452744,0.614125,0.559920
6,0.796900,1.288280,0.561633,0.452041,0.617000,0.561633



✓ Model saved to /kaggle/working/models/distilbert-base-multilingual-cased_focal_loss_final

✓ Version 2 (Focal Loss) Training Completed Successfully!


In [13]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import os

In [14]:
class PredictionDataset(Dataset):
    """Dataset for making predictions"""
    
    def __init__(self, texts, tokenizer, max_length=512):
        self.encodings = tokenizer(
            texts,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='pt'
        )
    
    def __len__(self):
        return len(self.encodings['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx]
        }

In [15]:
def make_predictions(
    model,
    tokenizer,
    test_texts,
    device,
    batch_size=32
):
    """Make predictions on test data"""
    
    print("\nMaking predictions...")
    
    dataset = PredictionDataset(test_texts, tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    all_predictions = []
    all_probs = []
    
    model.eval()
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            probs = torch.softmax(logits, dim=1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            
            if (i + 1) % 10 == 0:
                print(f"  Processed {(i + 1) * batch_size} samples...")
    
    print(f"✓ Predictions completed for {len(all_predictions)} samples\n")
    
    return np.array(all_predictions), np.array(all_probs)

In [16]:
def predict_with_model(
    test_csv_path,
    model_dir,
    model_name,
    output_csv_path,
    num_labels=15,
    batch_size=32,
    version_name="Model"
):
    """Make predictions using a trained model"""
    
    print(f"\n{'='*70}")
    print(f"PREDICTION: {version_name}")
    print(f"{'='*70}\n")
    
    # Check GPU
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"✓ GPU Available: {torch.cuda.get_device_name(0)}")
        print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB\n")
    else:
        device = torch.device('cpu')
        print(f"✗ GPU NOT available, using CPU\n")
    
    # Load test data
    print(f"Loading test data from: {test_csv_path}")
    test_df = pd.read_csv(test_csv_path)
    print(f"Test dataset shape: {test_df.shape}")
    print(f"Columns: {test_df.columns.tolist()}\n")
    
    # Load tokenizer
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Load model
    print(f"Loading model from: {model_dir}")
    if not os.path.exists(model_dir):
        print(f"✗ Model directory not found: {model_dir}")
        return None
    
    model = AutoModelForSequenceClassification.from_pretrained(
        model_dir,
        num_labels=num_labels
    )
    model.to(device)
    print("✓ Model loaded successfully\n")
    
    # Make predictions
    predictions, probabilities = make_predictions(
        model,
        tokenizer,
        test_df['text'].tolist(),
        device,
        batch_size=batch_size
    )
    
    # Create output dataframe
    output_df = pd.DataFrame({
        'id': test_df['id'],
        'label': predictions
    })
    
    # Add confidence scores
    output_df['confidence'] = np.max(probabilities, axis=1)
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    
    # Save predictions
    output_df.to_csv(output_csv_path, index=False)
    
    print(f"{'='*70}")
    print(f"✓ Predictions saved to: {output_csv_path}")
    print(f"{'='*70}\n")
    
    print("Sample predictions:")
    print(output_df.head(10))
    
    print(f"\nLabel distribution in predictions:")
    print(output_df['label'].value_counts().sort_index())
    
    print(f"\nPrediction statistics:")
    print(f"  - Mean confidence: {output_df['confidence'].mean():.4f}")
    print(f"  - Min confidence: {output_df['confidence'].min():.4f}")
    print(f"  - Max confidence: {output_df['confidence'].max():.4f}\n")
    
    return output_df

In [17]:
if __name__ == "__main__":
    
    # Configuration
    TEST_CSV_PATH = '/kaggle/input/ancient/test.csv'
    MODEL_NAME = 'distilbert/distilbert-base-multilingual-cased'
    NUM_LABELS = 15
    OUTPUT_DIR = '/kaggle/working/predictions'
    
    print(f"{'='*70}")
    print("DISTILBERT PREDICTION SCRIPT")
    print(f"{'='*70}\n")
    
    # ========================================================================
    # VERSION 1: WITHOUT PEFT
    # ========================================================================
    
    print(f"\n{'#'*70}")
    print("VERSION 1: WITHOUT PEFT")
    print(f"{'#'*70}")
    
    MODEL_DIR_NO_PEFT = '/kaggle/working/models/distilbert-base-multilingual-cased_focal_loss_final'
    OUTPUT_CSV_NO_PEFT = f'{OUTPUT_DIR}/predictions_focal_loss.csv'
    
    try:
        output_v1 = predict_with_model(
            test_csv_path=TEST_CSV_PATH,
            model_dir=MODEL_DIR_NO_PEFT,
            model_name=MODEL_NAME,
            output_csv_path=OUTPUT_CSV_NO_PEFT,
            num_labels=NUM_LABELS,
            batch_size=32,
            version_name="No PEFT"
        )
        print("✓ VERSION 1 COMPLETED SUCCESSFULLY!\n")
    except Exception as e:
        print(f"✗ Error in VERSION 1: {str(e)}")
        import traceback
        traceback.print_exc()

DISTILBERT PREDICTION SCRIPT


######################################################################
VERSION 1: WITHOUT PEFT
######################################################################

PREDICTION: No PEFT

✓ GPU Available: Tesla T4
✓ GPU Memory: 15.83 GB

Loading test data from: /kaggle/input/ancient/test.csv
Test dataset shape: (29914, 2)
Columns: ['id', 'text']

Loading tokenizer...
Loading model from: /kaggle/working/models/distilbert-base-multilingual-cased_focal_loss_final
✓ Model loaded successfully


Making predictions...
  Processed 320 samples...
  Processed 640 samples...
  Processed 960 samples...
  Processed 1280 samples...
  Processed 1600 samples...
  Processed 1920 samples...
  Processed 2240 samples...
  Processed 2560 samples...
  Processed 2880 samples...
  Processed 3200 samples...
  Processed 3520 samples...
  Processed 3840 samples...
  Processed 4160 samples...
  Processed 4480 samples...
  Processed 4800 samples...
  Processed 5120 samples...
  Proce